In [2]:
import pandas as pd
import ast

In [3]:
df = pd.read_csv('steam_games_limpio.csv', encoding='utf-8')
# Convertir la columna "release_date" al tipo datetime
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
# Eliminamos una columna
df.drop('Unnamed: 0', axis=1,inplace=True)

Se ingresa un año y devuelve una lista con los 5 géneros más ofrecidos en el orden correspondiente.

In [6]:
def genero(Año: str):
    df_filtered = df[df['release_date'].dt.year == int(Año)]

    df_filtered = df_filtered.dropna(subset=['genres'])

    df_filtered['genres'] = df_filtered['genres'].apply(ast.literal_eval)

    all_genres = [genre for sublist in df_filtered['genres'] for genre in sublist]

    genre_counts = pd.Series(all_genres).value_counts()

    top_genres_dict = genre_counts.head(5).to_dict()

    return top_genres_dict

# Ejemplo
print(genero(2017))

{'Indie': 5917, 'Action': 3517, 'Casual': 3141, 'Adventure': 2795, 'Strategy': 2255}


Se ingresa un año y devuelve una lista con los juegos lanzados en el año.

In [7]:
def juegos(Año:str):
    df_filtered = df[df['release_date'].dt.year == int(Año)]

    df_filtered = df_filtered.dropna(subset=['app_name'])

    return df_filtered['app_name']

# Ejemplo
juegos(2018)

0                 Lost Summoner Kitty
1                           Ironbound
5               Battle Royale Trainer
6        SNOW - All Access Basic Pass
7          SNOW - All Access Pro Pass
                     ...             
32049                           BAE 2
32050                    Kebab it Up!
32051                  Colony On Mars
32052        LOGistICAL: South Africa
32053                   Russian Roads
Name: app_name, Length: 96, dtype: object

Se ingresa un año y devuelve una lista con los 5 specs que más se repiten en el mismo en el orden correspondiente.

In [8]:
def specs(Año: str):
    df_filtered = df[df['release_date'].dt.year == int(Año)]

    df_filtered = df_filtered.dropna(subset=['specs'])

    df_filtered['specs'] = df_filtered['specs'].apply(ast.literal_eval)

    all_specs = [specs for sublist in df_filtered['specs'] for specs in sublist]

    specs_counts = pd.Series(all_specs).value_counts()

    top_specs_dict = specs_counts.head(5).to_dict()

    return top_specs_dict

# Ejemplo
print(specs(2018))

{'Single-player': 87, 'Steam Achievements': 42, 'Steam Cloud': 22, 'Full controller support': 20, 'Steam Trading Cards': 18}


Cantidad de juegos lanzados en un año con early access.

In [9]:
def earlyacces(Año:str):
    df_filtered = df[df['release_date'].dt.year == int(Año)]
    
    df_filtered = df_filtered.dropna(subset=['early_access'])

    cantidad_early_access = df_filtered['early_access'].sum()
    
    return cantidad_early_access

# Ejemplo
earlyacces(2017)

720

Según el año de lanzamiento, se devuelve una lista con la cantidad de registros que se encuentren categorizados con un análisis de sentimiento.

In [10]:
def sentiment(Año: str):
    df_filtered = df[df['release_date'].dt.year == int(Año)]

    df_filtered = df_filtered.dropna(subset=['sentiment'])

    df_filtered = df_filtered[~df_filtered['sentiment'].str.contains('user reviews')]

    sentiment_counts = df_filtered['sentiment'].value_counts()

    sentiment_dict = sentiment_counts.to_dict()
    
    return sentiment_dict

# Ejemplo
print(sentiment(2018))

{'Mixed': 6, 'Mostly Positive': 3, 'Very Positive': 3, 'Mostly Negative': 1}


Top 5 juegos según año con mayor metascore.

In [11]:
def metascore(Año: str):
    df_filtered = df[df['release_date'].dt.year == int(Año)]

    df_filtered = df_filtered.dropna(subset=['metascore'])

    df_sorted = df_filtered.sort_values(by='metascore', ascending=False)

    top_5_games = df_sorted.head(5)

    juegos_y_metascore = dict(zip(top_5_games['title'], top_5_games['metascore']))
    
    return juegos_y_metascore

# Ejemplo
metascore(2017)

{'OKAMI HD / 大神 絶景版': 93.0,
 'Divinity: Original Sin 2': 93.0,
 'Linelight': 90.0,
 'Bayonetta': 90.0,
 'F1™ 2017': 89.0}